In [153]:
# Importing necessary models
import warnings
warnings.filterwarnings('ignore')

import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
import time
%matplotlib inline
from bs4 import BeautifulSoup as bs
import requests
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from sqlalchemy import *
from sqlalchemy import create_engine
import calendar
import sqlite3 as sql
pd.options.display.float_format = '{:,.4f}'.format

init_notebook_mode(connected=True)

import os
main_dir = os.getcwd()

dbs_dir = 'C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Data\\DBs'

os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\\Modules\\DataCollection')

from yahoo_query import *

dbs_dir = 'D:\\Price Data'

os.chdir(dbs_dir)

# price_engine = create_engine('sqlite:///histprices.db', echo=False)

# inspector = inspect(price_engine)


## Getting SPX Constituents and Historical Compositions

In [309]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
soup = bs(requests.get(wiki_url,'lxml').text,'lxml')
sp_const = soup.select('table[class="wikitable sortable"]')[0]
changes = soup.select('table[class="wikitable sortable"]')[1]

sp_dict = {}
sp_headers = []
sp_exchange_dict = {}

for row in sp_const.find_all('th'):
    sp_headers.append(row.text.strip())
    sp_dict[row.text.strip()] = []
    
for row in sp_const.find_all('tr')[1:]:
    for i, col in enumerate(row.find_all('td')):
        if i == 0:
            url = col.find('a', href=True)['href']
            sp_exchange_dict[col.text.strip()] = [url[:url.find('.com') + len('.com')]]
            
        col_header = sp_headers[i]
        curr_val = col.text.strip()
        if 'Date' in col_header:
            try:
                curr_val = dt.datetime.strptime(curr_val, '%Y-%m-%d')
            except:
                curr_val = np.nan
        sp_dict[col_header].append(curr_val)
urls = pd.DataFrame(sp_exchange_dict).T
urls.columns = ['URL']
sp_const = pd.DataFrame(sp_dict).set_index('Symbol')
sp_const = sp_const[['Security','Date first added','Founded','GICS Sector','GICS Sub Industry']]
sp_const = pd.concat([sp_const,urls], axis = 1)

changes_headers = ['Date','Ticker_Added','Security_Added',
                   'Ticker_Removed']

changes_dict = {k:[] for k in changes_headers}


rowspan = 0
for row in changes.find_all('tr'):
       
    for i, col in enumerate(row.find_all('td')[:-1]):
        curr_val = col.text.strip()
        if len(row) == 12:
            if i == 0:
                curr_val = dt.datetime.strptime(curr_val, '%B %d, %Y')
                curr_date = curr_val
            if i == 4:
                continue
            changes_dict[changes_headers[i]].append(curr_val)
            
        else:
            if i == 0:
                changes_dict[changes_headers[i]].append(curr_date)
                changes_dict[changes_headers[1]].append(curr_val)
            elif i == 3:
                continue
            else:
                changes_dict[changes_headers[i + 1]].append(curr_val)
changes = pd.DataFrame(changes_dict)    

sp_500_hist = {dt.datetime.today().date():sp_const}

curr_const = sp_const.copy()

for change_date in changes.Date.drop_duplicates():
    curr_date = change_date.date()
    curr_changes = changes[changes.Date == change_date]
    curr_drops = curr_changes.Ticker_Added.tolist()
    curr_adds = curr_changes.Ticker_Removed.tolist()
    
    curr_adds = list(filter(lambda x: x != '', curr_adds))
    curr_drops = list(filter(lambda x: x != '', curr_drops))
    
    curr_rows = pd.DataFrame({'Symbol': curr_adds}).set_index('Symbol')
    
    curr_const = curr_const[curr_const.index.isin(curr_drops) == False]
    curr_const = pd.concat([curr_const, curr_rows], axis = 0)
    
    sp_500_hist[curr_date] = curr_const
    
    
all_sp500_names = pd.concat(list(sp_500_hist.values())).reset_index()[['Symbol','Security']].drop_duplicates()

KeyError: "['Symbol'] not in index"

In [58]:
prices_dfs = []
profiles_dfs = []

failed_prices = []
failed_profiles = []

start_time = time.time()

start_date = dt.datetime(2000,1,1)
for ticker in all_sp500_names.Symbol:
    curr_stock = yahoo_query(ticker.replace('.','-'), start_date)
    try:
        curr_stock.hist_prices_query()
        curr_prices = curr_stock.hist_prices[['{}_adjclose'.format(ticker)]]
        curr_prices.columns = [ticker]
        prices_dfs.append(curr_prices)
    except:
        failed_prices.append(ticker)
        continue
    
    try:
        curr_stock.full_info_query()
        curr_profile = curr_stock.profile
        profiles_dfs.append(curr_profile)
    except:
        failed_profiles.append(ticker)
        continue
    
end_time = time.time()
print("Completed in {} seconds.".format(end_time - start_time))

Completed in 1081.4660942554474 seconds.


In [113]:
cleaned_dfs = [df.reset_index().drop_duplicates().set_index('index') for df in prices_dfs]

sp_prices = pd.concat(cleaned_dfs, axis = 1)

sp_profiles = pd.concat(profiles_dfs, axis = 0).reset_index().drop_duplicates().set_index('index')

In [128]:
sp_const_engine = create_engine('sqlite:///sp500cons.db', echo = False)
sp_prices.to_sql('histprices', con = sp_const_engine, 
                 if_exists='replace', index_label = 'Date')
sp_profiles.to_sql('profiles', con = sp_const_engine, 
                   if_exists='replace', index_label = 'Symbol')

spx_const_dfs_lst = []

for k,v in sp_500_hist.items():
    curr_spx_df = v[['Security','Date first added']]
    curr_spx_df.columns = ['Name','Added_Date']
    curr_spx_df['Latest_Date'] = k
    spx_const_dfs_lst.append(curr_spx_df)
    
pd.concat(spx_const_dfs_lst, axis = 0).to_sql('histComponents', con = sp_const_engine, 
                                              if_exists='replace', index_label = 'Symbol')

In [152]:
cum_returns = (sp_prices.pct_change() + 1).cumprod(axis = 0).tail(1).T - 1
cum_returns.columns = ['cumret']
cum_returns.dropna().sort_values('cumret')

,cumret
CPWR,-0.9952
FTR,-0.9876
LIFE,-0.9615
AIG,-0.9516
JCP,-0.9078
S,-0.8901
AKS,-0.8444
ETFC,-0.8026
C,-0.7978
GNW,-0.7934


In [164]:
start_date = dt.datetime(2013,1,1)

spx = yahoo_query('^GSPC', start_date)
spx.hist_prices_query()

vix = yahoo_query('^VIX', start_date)
vix.hist_prices_query()

df = pd.concat([spx.hist_prices[['^GSPC_adjclose']],vix.hist_prices[['^VIX_adjclose']]], axis = 1)
change_df = pd.concat([df,df.pct_change()], axis = 1)
change_df.columns = ['SPX','VIX','SPX_ret','VIX_ret']

In [168]:
change_df[(change_df.SPX_ret <= -0.02) &
          (change_df.SPX_ret >= -0.025)].sort_values('VIX_ret', ascending = False)

,SPX,VIX,SPX_ret,VIX_ret
2013-04-15,"1,552.3600",17.2700,-0.0230,0.4320
2016-09-09,"2,127.8101",17.5000,-0.0245,0.3989
2015-06-29,"2,057.6399",18.8500,-0.0209,0.3445
2014-01-24,"1,790.2900",18.1400,-0.0209,0.3174
2018-02-02,"2,762.1299",17.3100,-0.0212,0.2851
2015-08-20,"2,035.7300",19.1400,-0.0211,0.2551
2014-10-09,"1,928.2100",18.7600,-0.0207,0.2416
2016-01-07,"1,943.0900",24.9900,-0.0237,0.2137
2018-04-02,"2,581.8799",23.6200,-0.0223,0.1828
2014-02-03,"1,741.8900",21.4400,-0.0228,0.1646


In [337]:
def get_gurufocus(ticker, url):
    from selenium import webdriver
    from selenium.common.exceptions import TimeoutException
    
    browser = webdriver.Chrome(executable_path=r"C:\Users\Fang\Desktop\Python Trading\Trading\chromedriver.exe")
    
    if url == 'https://www.nyse.com':
        gurufocus = 'https://www.gurufocus.com/stock/NYSE:{}/dcf'.format(ticker)
    elif url == 'http://www.nasdaq.com':
        gurufocus = 'https://www.gurufocus.com/stock/NAS:{}/dcf'.format(ticker)
    else:
        gurufocus = 'https://www.gurufocus.com/stock/CBOE:{}/dcf'.format(ticker)
    
    browser.get(gurufocus)
    dcfs = [element.text for element in browser.find_elements_by_xpath("//div[contains(@class, 'el-form-item el-form-item--mini')]")]
    
    browser.quit()
    
    importants = list(filter(lambda x: 'Fair Value :' in x, dcfs)) + list(filter(lambda x: 'Margin Of Safety :' in x, dcfs)) + list(filter(lambda x: 'Growth Rate :' in x and 'Terminal' not in x, dcfs))

    return importants

In [378]:
start_time = time.time()

gurus_df = urls.copy()
del gurus_df['URL']
gurus_df['SpotPrice'] = np.nan
gurus_df['FairValue'] = np.nan
gurus_df['MarginOfSafety'] = np.nan
gurus_df['ExpectedGrowthRate'] = np.nan

i = 1
for ticker, row in urls.iterrows():
    
    try:
        dcfs = get_gurufocus(ticker, row.URL)
    except:
        None
    
    try:
        gurus_df.loc[ticker,'FairValue'] = float(dcfs[0].split(':')[-1].replace('$','').replace('%','').strip())
    except:
        None
    
    try:
        gurus_df.loc[ticker,'MarginOfSafety'] = float(dcfs[1].split(':')[-1].replace('$','').replace('%','').strip())/100
    except:
        None
    
    try:
        gurus_df.loc[ticker,'ExpectedGrowthRate'] = float(dcfs[2].split(':')[-1].replace('$','').replace('%','').strip())/100
    except:
        None
    
    print("Completed {0}% for {1}".format(str(round(i*100/len(gurus_df),2)),ticker))
    i += 1

gurus_df['SpotPrice'] = gurus_df['FairValue']*(1 - gurus_df['MarginOfSafety'])
gurus_df.to_csv('SPX_const_dcf.csv')
end_time = time.time()
print("Completed in {} seconds.".format(end_time - start_time))

Completed 0.2% for A
Completed 0.4% for AAL
Completed 0.59% for AAP
Completed 0.79% for AAPL
Completed 0.99% for ABBV
Completed 1.19% for ABC
Completed 1.39% for ABMD
Completed 1.58% for ABT
Completed 1.78% for ACN
Completed 1.98% for ADBE
Completed 2.18% for ADI
Completed 2.38% for ADM
Completed 2.57% for ADP
Completed 2.77% for ADS
Completed 2.97% for ADSK
Completed 3.17% for AEE
Completed 3.37% for AEP
Completed 3.56% for AES
Completed 3.76% for AFL
Completed 3.96% for AGN
Completed 4.16% for AIG
Completed 4.36% for AIV
Completed 4.55% for AIZ
Completed 4.75% for AJG
Completed 4.95% for AKAM
Completed 5.15% for ALB
Completed 5.35% for ALGN
Completed 5.54% for ALK
Completed 5.74% for ALL
Completed 5.94% for ALLE
Completed 6.14% for ALXN
Completed 6.34% for AMAT
Completed 6.53% for AMD
Completed 6.73% for AME
Completed 6.93% for AMG
Completed 7.13% for AMGN
Completed 7.33% for AMP
Completed 7.52% for AMT
Completed 7.72% for AMZN
Completed 7.92% for ANET
Completed 8.12% for ANSS
Comple

Completed 65.54% for MU
Completed 65.74% for MXIM
Completed 65.94% for MYL
Completed 66.14% for NBL
Completed 66.34% for NCLH
Completed 66.53% for NDAQ
Completed 66.73% for NEE
Completed 66.93% for NEM
Completed 67.13% for NFLX
Completed 67.33% for NI
Completed 67.52% for NKE
Completed 67.72% for NKTR
Completed 67.92% for NLSN
Completed 68.12% for NOC
Completed 68.32% for NOV
Completed 68.51% for NRG
Completed 68.71% for NSC
Completed 68.91% for NTAP
Completed 69.11% for NTRS
Completed 69.31% for NUE
Completed 69.5% for NVDA
Completed 69.7% for NWL
Completed 69.9% for NWS
Completed 70.1% for NWSA
Completed 70.3% for O
Completed 70.5% for OKE
Completed 70.69% for OMC
Completed 70.89% for ORCL
Completed 71.09% for ORLY
Completed 71.29% for OXY
Completed 71.49% for PAYX
Completed 71.68% for PBCT
Completed 71.88% for PCAR
Completed 72.08% for PEG
Completed 72.28% for PEP
Completed 72.48% for PFE
Completed 72.67% for PFG
Completed 72.87% for PG
Completed 73.07% for PGR
Completed 73.27% for 

In [431]:
summaries_df = urls.copy()
del summaries_df['URL']
summaries_df['FinancialStrengthScore'] = np.nan
summaries_df['ProfitAndGrowthScore'] = np.nan

start_time = time.time()

i = 1

for ticker, row in urls.iterrows():
    try:
        if row.URL == 'https://www.nyse.com':
            gurufocus = 'https://www.gurufocus.com/stock/NYSE:{}/summary'.format(ticker)
        elif row.URL == 'http://www.nasdaq.com':
            gurufocus = 'https://www.gurufocus.com/stock/NAS:{}/summary'.format(ticker)
        soup = bs(requests.get(gurufocus).text,'lxml')
        summaries = [x.text.strip() for x in soup.select('h2[class="page-title"]')][:2]
        summaries = [int(x.split(':')[-1].strip().split('/')[0]) for x in summaries]
    except:
        continue
    
    try:
        summaries_df.loc[ticker,'FinancialStrengthScore'] = summaries[0]
    except:
        None
    
    try:
        summaries_df.loc[ticker,'ProfitAndGrowthScore'] = summaries[1]
    except:
        None
    
    print("Completed {0}% for {1}".format(str(round(i*100/len(gurus_df),2)),ticker))
    i += 1
end_time = time.time()
print("Completed in {} seconds.".format(end_time - start_time))

Completed 0.2% for A
Completed 0.4% for AAL
Completed 0.59% for AAP
Completed 0.79% for AAPL
Completed 0.99% for ABBV
Completed 1.19% for ABC
Completed 1.39% for ABMD
Completed 1.58% for ABT
Completed 1.78% for ACN
Completed 1.98% for ADBE
Completed 2.18% for ADI
Completed 2.38% for ADM
Completed 2.57% for ADP
Completed 2.77% for ADS
Completed 2.97% for ADSK
Completed 3.17% for AEE
Completed 3.37% for AEP
Completed 3.56% for AES
Completed 3.76% for AFL
Completed 3.96% for AGN
Completed 4.16% for AIG
Completed 4.36% for AIV
Completed 4.55% for AIZ
Completed 4.75% for AJG
Completed 4.95% for AKAM
Completed 5.15% for ALB
Completed 5.35% for ALGN
Completed 5.54% for ALK
Completed 5.74% for ALL
Completed 5.94% for ALLE
Completed 6.14% for ALXN
Completed 6.34% for AMAT
Completed 6.53% for AMD
Completed 6.73% for AME
Completed 6.93% for AMG
Completed 7.13% for AMGN
Completed 7.33% for AMP
Completed 7.52% for AMT
Completed 7.72% for AMZN
Completed 7.92% for ANET
Completed 8.12% for ANSS
Comple

Completed 65.54% for MXIM
Completed 65.74% for MYL
Completed 65.94% for NBL
Completed 66.14% for NCLH
Completed 66.34% for NDAQ
Completed 66.53% for NEE
Completed 66.73% for NEM
Completed 66.93% for NFLX
Completed 67.13% for NI
Completed 67.33% for NKE
Completed 67.52% for NKTR
Completed 67.72% for NLSN
Completed 67.92% for NOC
Completed 68.12% for NOV
Completed 68.32% for NRG
Completed 68.51% for NSC
Completed 68.71% for NTAP
Completed 68.91% for NTRS
Completed 69.11% for NUE
Completed 69.31% for NVDA
Completed 69.5% for NWL
Completed 69.7% for NWS
Completed 69.9% for NWSA
Completed 70.1% for O
Completed 70.3% for OKE
Completed 70.5% for OMC
Completed 70.69% for ORCL
Completed 70.89% for ORLY
Completed 71.09% for OXY
Completed 71.29% for PAYX
Completed 71.49% for PBCT
Completed 71.68% for PCAR
Completed 71.88% for PEG
Completed 72.08% for PEP
Completed 72.28% for PFE
Completed 72.48% for PFG
Completed 72.67% for PG
Completed 72.87% for PGR
Completed 73.07% for PH
Completed 73.27% for 

In [434]:
df = pd.concat([gurus_df, summaries_df], axis = 1)

In [439]:
df[(df.MarginOfSafety > 0) &
   (df.FinancialStrengthScore > 7) &
   (df.ProfitAndGrowthScore > 7)].sort_values('MarginOfSafety', ascending = False)

,SpotPrice,FairValue,MarginOfSafety,ExpectedGrowthRate,FinancialStrengthScore,ProfitAndGrowthScore
MU,37.3744,116.5400,0.6793,-0.1468,8.0000,9.0000
SWKS,76.6250,170.2400,0.5499,0.0782,9.0000,9.0000
RHI,55.6504,105.9000,0.4745,0.1022,8.0000,8.0000
FFIV,143.4415,235.1500,0.3900,0.1250,9.0000,9.0000
TROW,101.1322,152.8600,0.3384,0.0847,9.0000,8.0000
AOS,48.1006,72.3100,0.3348,0.1382,8.0000,8.0000
CTSH,56.8777,74.7800,0.2394,0.1153,8.0000,8.0000
IPGP,142.4157,184.1900,0.2268,0.1611,8.0000,8.0000
ROST,93.4462,116.5600,0.1983,0.1606,8.0000,8.0000
